<a href="https://colab.research.google.com/github/thadeupenna/visualizacao/blob/master/COVID_19_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Séries Temporais da COVID-19

## Médias Móveis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from urllib.request import Request, urlopen
import datetime as dt
import io
import gzip

In [ ]:
url = "https://data.brasil.io/dataset/covid19/caso_full.csv.gz"

request = Request(url, headers={"User-Agent": "python-urllib"})
response = urlopen(request)
data = gzip.decompress(response.read()).decode("utf-8")
df = pd.read_csv(io.StringIO(data))

In [ ]:
df.head()

In [ ]:
brasil = df[df.place_type == 'state'].copy()

brasil_novos_casos = brasil.groupby('date').new_confirmed

In [ ]:
brasil_novos_casos.sum().plot(kind='bar', figsize=(20,8))

In [ ]:
novos_casos_mm = brasil_novos_casos.sum().rolling(15, center=True).mean()

In [ ]:
novos_casos_mm.plot(kind='bar', figsize=(20,8))

### Estaríamos no pico ??? 

In [ ]:
novos_casos_mm = brasil_novos_casos.sum().rolling(14, center=True, win_type='gaussian').mean(std=3)

In [ ]:
plt.style.use("ggplot")
novos_casos_mm.plot(kind='bar', figsize=(20,8));

In [ ]:
novos_casos_mm

In [ ]:
type(novos_casos_mm.index[3])

In [ ]:
lista_de_datas = novos_casos_mm.index.to_list()
casos = novos_casos_mm.values
datas = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in lista_de_datas]
print(list(zip(datas,casos))[:10])

# Animações com MatplotLib

In [ ]:
from matplotlib.animation import FuncAnimation
from matplotlib import rc

fig,ax = plt.subplots(1,1, figsize=(14,8))
plt.close()
line, = ax.plot_date([], [], 'r-', lw=3, xdate=True)
#  ax.plot retorna uma lista Line2D, por isso a vírgula

ax.set_ylim(0,40000)
ax.set_xlim(datas[0],datas[-1])
ax.xaxis.set_major_locator(plt.MultipleLocator(14))

def init():
  line, = ax.plot_date([], [])
  return line,

def animate(i):
  x = datas[:i]
  y = casos[:i]
  line.set_data(x, y)
  return line,

anim = FuncAnimation(fig, animate, init_func=init, frames=len(datas), interval=5, blit=True)

rc('animation', html='jshtml')
anim

In [ ]:
!apt-get update && apt-get install ffmpeg

In [ ]:
from matplotlib.animation import FuncAnimation, writers, FFMpegWriter
from matplotlib import rc

fig,ax = plt.subplots(1,1, figsize=(14,8))
plt.close()
line = ax.bar([], [])

ax.set_ylim(0,40000)
ax.set_xlim(datas[0],datas[-1])
ax.xaxis.set_major_locator(plt.MultipleLocator(14))
anotacao = ax.text(datas[10],35000,'', fontsize=30)

def init():
  line = ax.bar([], [])
  return line

def animate(i):
  x = datas[:i]
  y = casos[:i]

  anotacao.set_x(datas[i])
  if (casos[i] != casos[i]):
    yp = 10
    anotacao.set_text('')
  else:
    yp = casos[i]  
    anotacao.set_y(yp)
    anotacao.set_text(f'{casos[i]:.0f}')

  line = ax.bar(x,y, color='red')
  return line

anim = FuncAnimation(fig, animate, init_func=init, frames=len(datas), interval=100, blit=True)

rc('animation', html='jshtml', ffmpeg_path = '/usr/bin/ffmpeg')
anim

writer = FFMpegWriter(fps=20)
anim.save("movie.mp4", writer=writer, dpi=100)

# anim.save('covid.gif', writer='pillow')

# Merge Dataframes


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive
%cd 'My Drive/Colab Notebooks'

In [ ]:
municipios = pd.read_csv('municipios.csv')

In [ ]:
municipios.head()

In [ ]:
df.head()

### Critério

In [ ]:
criterio = 'last_available_death_rate'

In [ ]:
casos_estudados = df[ (df.place_type=='city') & (df.last_available_date == '2020-07-18')].copy()

In [ ]:
casos_estudados.sort_values(criterio,ascending=False, inplace=True)

In [ ]:
casos_estudados[:100]

In [ ]:
casos_estudados.columns

In [ ]:
casos_estudados_limpos =  casos_estudados[['city','city_ibge_code', 'estimated_population_2019', 'state', criterio]].copy()

In [ ]:
casos_estudados_limpos.head()

In [ ]:
municipios.head()

In [ ]:
casos_novos_municipios = pd.merge(casos_estudados_limpos, municipios[['codigo_ibge','latitude', 'longitude']], 
                                  left_on='city_ibge_code', right_on='codigo_ibge')
casos_novos_municipios.head()

In [ ]:
casos_novos_municipios.drop('codigo_ibge', axis=1)

In [ ]:
casos_novos_municipios.drop( 'codigo_ibge', axis=1, inplace=True )

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.scatter(casos_novos_municipios.longitude[:100],casos_novos_municipios.latitude[:100],s=casos_novos_municipios[criterio][:100]*300,
           alpha=0.5)
ax.set_aspect(1)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
img = plt.imread('mapa-do-brasil-estados-branco-comlegenda.jpg')
ax.imshow(img, extent=[casos_novos_municipios.longitude.min(),casos_novos_municipios.longitude.max(),
                      casos_novos_municipios.latitude.min(),casos_novos_municipios.latitude.max()])
ax.scatter(casos_novos_municipios.longitude[:100],casos_novos_municipios.latitude[:100], s=casos_novos_municipios[criterio][:100]*300, alpha=0.5)
ax.set_aspect(1)


In [ ]:
import folium

In [ ]:
mapa = folium.Map(location=[-15.788497,-47.879873], zoom_start=4)
for la,lo in zip( casos_novos_municipios.latitude[:100], casos_novos_municipios.longitude[:100] ):
    folium.Marker([la, lo]).add_to(mapa)
display(mapa)


In [ ]:
mapa = folium.Map(location=[-15.788497,-47.879873], zoom_start=4)
for la,lo,rd in zip( casos_novos_municipios.latitude[:100], casos_novos_municipios.longitude[:100], 
                    casos_novos_municipios[criterio][:100] ):
    folium.CircleMarker([la, lo],radius=rd*10).add_to(mapa)
display(mapa)
